In [83]:
import pandas as pd
import plotly.express as px

from sklearn import preprocessing

import plotly.graph_objects as go

import numpy as np
import pandas as pd
import scipy

from scipy import signal


In [125]:
df_minutes = pd.read_csv(f'datasources\\transformer_sentiment_analysis\\sentiments\\minutes.csv')
df_minutes

,Unnamed: 0.1,Unnamed: 0,date,contents,speaker,title,fin_sentiment,gen_sentiment
0,0,0,1993-02-03,A meeting of the Federal Open Market Committee...,Alan Greenspan,FOMC Meeting Minutes,0.431132,0.913638
1,1,1,1993-03-23,A meeting of the Federal Open Market Committee...,Alan Greenspan,FOMC Meeting Minutes,0.326468,0.771585
2,2,2,1993-05-18,A meeting of the Federal Open Market Committee...,Alan Greenspan,FOMC Meeting Minutes,0.258086,0.787030
3,3,3,1993-07-07,A meeting of the Federal Open Market Committee...,Alan Greenspan,FOMC Meeting Minutes,0.359459,0.991968
4,4,4,1993-08-17,A meeting of the Federal Open Market Committee...,Alan Greenspan,FOMC Meeting Minutes,0.332222,0.891136
...,...,...,...,...,...,...,...,...
224,224,224,2022-03-16,"The Federal Reserve, the central bank of the U...",other,FOMC Meeting Minutes,0.391951,0.954031
225,225,225,2022-05-04,"The Federal Reserve, the central bank of the U...",other,FOMC Meeting Minutes,0.455136,0.983596
226,226,226,2022-06-15,"The Federal Reserve, the central bank of the U...",other,FOMC Meeting Minutes,0.322443,0.917143
227,227,227,2022-07-27,"The Federal Reserve, the central bank of the U...",other,FOMC Meeting Minutes,0.277176,0.992117


In [148]:
df= pd.read_csv(f"datasources\master_dataset.csv")
# df = df[["date", "rate", "uncertainty",	"litigious", "strong_modal",	"weak_modal", 	"constraining", "optimistic"]].dropna()
df.head()

,date,rate,uncertainty,litigious,strong_modal,weak_modal,constraining,optimistic,rsi_14,stochrsi_14,...,W.FISH,WGTS.AGRI,WGTS.BRENT,WGTS.COAL,WGTS.FISH,WGTS.FOPR,WGTS.MTLS,WGTS.NATURALGAS,WGTS.WCC,WGTS.WTI
0,1972-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,30.096644,0.133935,1.183078,0.920545,30.444547,20.618571,2.926147,4.404203,9.272329
1,1972-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,30.096644,0.133935,1.183078,0.920545,30.444547,20.618571,2.926147,4.404203,9.272329
2,1972-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,30.096644,0.133935,1.183078,0.920545,30.444547,20.618571,2.926147,4.404203,9.272329
3,1972-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,30.096644,0.133935,1.183078,0.920545,30.444547,20.618571,2.926147,4.404203,9.272329
4,1972-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,30.096644,0.133935,1.183078,0.920545,30.444547,20.618571,2.926147,4.404203,9.272329


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18560 entries, 0 to 18559
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 18560 non-null  object 
 1   rate                 10599 non-null  float64
 2   uncertainty          10865 non-null  float64
 3   litigious            10865 non-null  float64
 4   strong_modal         10865 non-null  float64
 5   weak_modal           10865 non-null  float64
 6   constraining         10865 non-null  float64
 7   optimistic           10865 non-null  float64
 8   rsi_14               10586 non-null  float64
 9   stochrsi_14          10573 non-null  float64
 10  macd_12_26           10574 non-null  float64
 11  adx_14               10599 non-null  float64
 12  williams_%R          10586 non-null  float64
 13  cci                  10580 non-null  float64
 14  atr                  10599 non-null  float64
 15  utlimate_oscillator  10571 non-null 

In [ ]:
us_indices_df = pd.concat([ df.set_index('date'), df_minutes.set_index('date')], axis=1, join='inner')

In [41]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

norm_df = normalize( df[[ "rate", "uncertainty",	"litigious", "strong_modal",	"weak_modal", 	"constraining", "optimistic"]])

norm_df

,rate,uncertainty,litigious,strong_modal,weak_modal,constraining,optimistic
7693,0.492698,0.353448,0.367347,0.222222,0.405405,0.409091,0.753521
7694,0.496564,0.353448,0.367347,0.222222,0.405405,0.409091,0.753521
7695,0.488832,0.353448,0.367347,0.222222,0.405405,0.409091,0.753521
7698,0.488832,0.353448,0.367347,0.222222,0.405405,0.409091,0.753521
7699,0.490550,0.353448,0.367347,0.222222,0.405405,0.409091,0.753521
...,...,...,...,...,...,...,...
18551,0.622709,0.172414,0.122449,0.333333,0.324324,0.045455,0.799296
18552,0.624427,0.172414,0.122449,0.333333,0.324324,0.045455,0.799296
18555,0.631586,0.172414,0.122449,0.333333,0.324324,0.045455,0.799296
18556,0.633018,0.172414,0.122449,0.333333,0.324324,0.045455,0.799296


In [57]:
features = ['rate', 'uncertainty']

fig = px.scatter_matrix(
    df,
    dimensions=features,
    # color="date",
    width=2000, height=1000
)
fig.update_traces(diagonal_visible=False)
fig.show()

e:\Personal\GitHub\CHAMOAnalyticsDatathon2022\venv\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [141]:
import plotly.express as px

fig = px.line( x=df['date'], y=norm_df["rate"], title="Historique de sentiments des textes du FOMC et du taux d'échange USD/CAD",  width=2000, height=1000,
labels={
                     "x": "Date",
                     "y": "Taux d'échange USD/CAD normalized [0,1]",
                 })
# fig.add_scatter(x=df['date'], y=norm_df['uncertainty'], mode='lines', name="uncertainty")
# fig.add_scatter(x=df['date'], y=norm_df['litigious'], mode='lines', name="litigious")
# fig.add_scatter(x=df['date'], y=norm_df['strong_modal'], mode='lines', name="strong_modal")
# fig.add_scatter(x=df['date'], y=norm_df['weak_modal'], mode='lines', name="weak_modal")
# fig.add_scatter(x=df['date'], y=norm_df['constraining'], mode='lines', name="constraining")
# fig.add_scatter(x=df['date'], y=norm_df['optimistic'], mode='lines', name="optimistic")
# fig.add_trace(go.Scatter(
#     x=df['date'],
#     y=signal.savgol_filter(norm_df['uncertainty'],
#                            100, # window size used for filtering
#                            5), # order of fitted polynomial
#     mode='markers',
#     marker=dict(
#         size=6,
#         # color='mediumpurple',
#         # symbol='triangle-up'
#     ),
#     name='uncertainty'
# ))
# fig.add_trace(go.Scatter(
#     x=df['date'],
#     y=signal.savgol_filter(norm_df['litigious'],
#                            100, # window size used for filtering
#                            5), # order of fitted polynomial
#     mode='markers',
#     marker=dict(
#         size=6,
#         # color='mediumpurple',
#         # symbol='triangle-up'
#     ),
#     name='litigious'
# ))
fig.add_trace(go.Scatter(
    x=df['date'],
    y=signal.savgol_filter(norm_df['strong_modal'],
                           100, # window size used for filtering
                           1), # order of fitted polynomial
    mode='markers',
    marker=dict(
        size=6,
        # color='mediumpurple',
        # symbol='triangle-up'
    ),
    name='Normalized Strong_Modal word count'
))
fig.add_trace(go.Scatter(
    x=df['date'],
    y=signal.savgol_filter(norm_df['weak_modal'],
                           100, # window size used for filtering
                           1), # order of fitted polynomial
    mode='markers',
    marker=dict(
        size=6,
        # color='mediumpurple',
        # symbol='triangle-up'
    ),
    name='Normalized Weak_Modal  word count'
))
fig.add_trace(go.Scatter(
    x=df['date'],
    y=signal.savgol_filter(norm_df['constraining'],
                           100, # window size used for filtering
                            1), # order of fitted polynomial
    mode='markers',
    marker=dict(
        size=6,
        # color='mediumpurple',
        # symbol='triangle-up'
    ),
    name='Normalized Constraining word count'
))
fig.add_trace(go.Scatter(
    x=df['date'],
    y=signal.savgol_filter(norm_df['optimistic'],
                           100, # window size used for filtering
                           1), # order of fitted polynomial
    mode='markers',
    marker=dict(
        size=6,
        # color='mediumpurple',
        # symbol='triangle-up'
    ),
    name='Normalized Optimistic score'
))

# fig.add_trace(go.Scatter(
#     x=df['date'],
#     y=signal.savgol_filter(df_minutes['fin_sentiment'],
#                            100, # window size used for filtering
#                            1), # order of fitted polynomial
#     mode='markers',
#     marker=dict(
#         size=6,
#         # color='mediumpurple',
#         # symbol='triangle-up'
#     ),
#     name='Normalized Optimistic score'
# ))


fig.show()

In [145]:

fig = px.line( x=df['date'], y=norm_df['rate'], title="Historique de sentiments des textes du FOMC et du taux d'échange USD/CAD (bleu)",  width=2000, height=1000,
labels={
                     "y": "Date",
                     "x": "Taux d'échange USD/CAD",
                 })

fig.add_scatter(x=df_minutes['date'], y=df_minutes['gen_sentiment'], mode='lines', name="Sentiment General")
fig.add_scatter(x=df_minutes['date'], y=df_minutes['fin_sentiment'], mode='lines', name="Sentiment Financier")
# fig.add_scatter(x=df['date'], y=norm_df['rate'], mode='lines', name="taux d'échange USD/CAD")


fig.show()

In [115]:
import plotly.express as px
from sklearn.decomposition import PCA

# df = px.data.iris()
# features = ["sepal_width", "sepal_length", "petal_width", "petal_length"]

features = [  "uncertainty",	"litigious", "strong_modal",	"weak_modal", 	"constraining", "optimistic"]

pca = PCA()
components = pca.fit_transform(norm_df[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    # color=df["species"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

e:\Personal\GitHub\CHAMOAnalyticsDatathon2022\venv\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [ ]:
import plotly.express as px

fig = px.line( x=df['date'], y=components["rate"],   width=2000, height=2000)
# fig.add_scatter(x=df['date'], y=norm_df['uncertainty'], mode='lines', name="uncertainty")
# # fig.add_scatter(x=df['date'], y=norm_df['litigious'], mode='lines', name="litigious")
# # fig.add_scatter(x=df['date'], y=norm_df['strong_modal'], mode='lines', name="strong_modal")
# # fig.add_scatter(x=df['date'], y=norm_df['weak_modal'], mode='lines', name="weak_modal")
# # fig.add_scatter(x=df['date'], y=norm_df['constraining'], mode='lines', name="constraining")
# fig.add_scatter(x=df['date'], y=norm_df['optimistic'], mode='lines', name="optimistic")

fig.show()